In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Loading libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Loading the dataset 
online = pd.read_csv('../input/onlineretail/OnlineRetail.csv',  engine='python')
online.head()

In [ ]:
#Adding columns: invoice_value,Year, Month, Day and Date to online dataframe
online['invoice_value'] = online['Quantity']*online['UnitPrice']
online['Year'] = pd.DatetimeIndex(online['InvoiceDate']).year
online['Month'] = pd.DatetimeIndex(online['InvoiceDate']).month
online['Day'] =  pd.DatetimeIndex(online['InvoiceDate']).day
online['Date'] = pd.DatetimeIndex(online['InvoiceDate']).date
online.tail()

In [ ]:
#Monthly revenue is sum up all 'invoice_value' in a month
monthly = online.groupby(['Year', 'Month'])['invoice_value'].sum()
monthly = monthly.to_frame()
monthly = monthly.set_index(pd.Index(monthly.index))

In [ ]:
#Plotting the monthly revenue
sns.set_style('whitegrid')
h = sns.barplot(x = 'invoice_value', y = monthly.index, data = monthly, color = 'lightseagreen')
h.figure.set_size_inches(10, 6)
h.set_xlabel('Revenue in Millions GBP', size = 12)
h.set_ylabel('Month of Year', size = 12)
h.set_title('Monthly Revenue', size = 14, y = 1.05)
plt.show()

In [ ]:
#Revenue by countries is sum up of all 'invoice_value' for each country.
revenue_per_country = online.groupby(['Country'])['invoice_value'].sum().sort_values(ascending=False)
revenue_per_country = revenue_per_country.to_frame()
revenue_per_country.head()

In [ ]:
#Plotting countries revenue. As the domestic revenue is 30 times higher than the 2nd highest foreign market - Netherlands, 
#So we exclude the UK from our chart, to see the values in other countries. 

#Excuding the UK from the dataset
country = revenue_per_country[revenue_per_country.index != 'United Kingdom']

#Plotting 

g = sns.barplot(x = 'invoice_value', y = country.index , data = country, color = 'darkcyan')
g.figure.set_size_inches(12, 10)
g.set_xlabel('Revenue in GBP', size = 12)
g.set_ylabel('Country', size = 12)
g.set_title('Revenue in Countries outside the UK', size = 14, y = 1.05)
plt.show()

In [ ]:
#The total revenue 
rev = online['invoice_value'].sum()
print('Total revenue of the business represented in the dataset is', np.round(rev,0), ' GBP')

### An overview of the dataset and the business
The dataset recorded sales transactions from 1/12/2010 to 9/12/2011, 12 months and 9 days. The total revenue for all recorded time is 9,74 million GBP(assumption),  for the last 9 days of the dataset is about 0.42 million GBP, so annual income for this business recorded is 9.3 million GBP. 

Nov is the busiest month with the highest earnings, following by October and September. Revenue from earlier months of the year was much lower than later months. 

With only 9 days in Dec 2011, revenue reached 0.42 million GBP, from my prediction, it could get up to 1.5 million by the end of the month due to holiday season. If that prediction is accurate, the earnings of Dec 2011 would be doubled of Dec the previous year. Another assumption is, this was a new business. 

The foreign market of the business is surrounding European countries and Australia (!). However, it only took 8.5% total revenue of the company.

For further analysis, I will exclude Dec 2011 from the dataset

In [ ]:
#Excluding Dec, 2011 
df1 = online[(online['Year']==2010)]
df2 = online[(online['Month']!=12)]
df = pd.concat([df1,df2])


In [ ]:
#Groupping revenue per invoice 
invoice = df.groupby(['Month','InvoiceNo'])['invoice_value'].sum()
invoice = invoice.to_frame()
invoice = invoice.reset_index()

In [ ]:
#Plotting the invoice distribution for each month
g = sns.boxplot(x = 'Month',y= 'invoice_value', data = invoice, order = [12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
g.set_title('Invoice values distribution over months', size = 16, y = 1.05)
g.set_xlabel('Month', size = 14)
g.set_ylabel('Invoice Value in GBP', size = 14)
g.figure.set_size_inches(15, 8)
plt.show()

In [ ]:
#There are big outliners in pervious plots, so we will slice into the smaller range with more density value
g = sns.boxplot(x = 'Month',y= 'invoice_value', data = invoice, order = [12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
g.set_title('Invoice values distribution over months', size = 16, y = 1.05)
g.set_xlabel('Month', size = 14)
g.set_ylabel('Invoice Value in GBP', size = 14)
g.figure.set_size_inches(15, 12)
g.set_ylim(-5000, 10000)
plt.show()

In [ ]:
#Groupping revenue by day
daily = df.groupby(['Month', 'Day'])['invoice_value'].sum()
daily = daily.to_frame()
daily = daily.reset_index()

#Plotting daily revenue distribution over months
g = sns.boxplot(x = 'Month',y= 'invoice_value', data = daily, order = [12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
g.figure.set_size_inches(15, 8)
g.set_title('Daily revenue distribution by month', size = 14, y = 1.05)
g.set_ylabel('Daily Revenue - GBP', size = 12)
g.set_xlabel('Month', size = 12)
plt.show()

### Customer Analysis

The first boxplot shows that dataset has numbers of extreme outliners. It looks like there were a couple of big contracts (or input mistakes) they were likely to make but then got cancelled. 

Many of other outliners dropped to the range of 20K GBP, which is way larger than an average bill of any normal person to buy for their home. In other words, they do business with other business as well. 

Zooming into the smaller range (The 2nd boxplot), the number of outliners still significant. Even though the invoice values are not much varied over months, but the daily revenue distribution over months (Box plot 3rd) are significantly different from months to months. Concretely, these outliners have effected strongly on daily income. There is not enough data to conclude, but I can predict that B2B had a big impact on this business. Taking good care of them would be the core strategy. 

### Calculating CLTV

There are multiple methods to calculate CLTV to use in specific purposes. For this business and the dataset available, I will estimate CLTV by the granular formula, which is driven by how frequency a customer purchases with the business, what are their average value per purchase and the estimate customer lifespan. 

Customer lifespan is a critical metric which heavily depended on the business model itself. When it comes to customer lifespan, it’s important to understand the difference between being a contractual and non-contractual business. Most online-stores are non-contractual businesses, it means, when a transaction is made, it is effectively over. From this article of https://www.shopify.ca/blog/customer-lifetime-value, I estimate the life-span of customer for this business is 24 months. 

In [ ]:
#Granular CLTV Calculation 

#Calculate average revenue per invoice 
revenue_per_purchase = online.groupby(['InvoiceNo'])['invoice_value'].mean().mean()

#Calculate average number unique invoices per customer per month
freg = online.groupby(['CustomerID', 'Month'])['InvoiceNo'].nunique().mean()

#Define customer life-spand to 24 months
lifespand_months = 24

#Calculate granular CLV
cltv_granular = revenue_per_purchase*freg*lifespand_months

# Print granular CLV value
print('Average granular CLV is {:.1f} USD'.format(cltv_granular))

In [ ]:
#Using Linear Regression to predict next month's purchase
#Inspecting for sales distribution by months
df2.groupby(['Month']).size()

In [ ]:
df2['Date'] = pd.to_datetime(df2['Date'])

#Exclude target variable
online_X=df2[df2['Month']!= 11] 

# Define the snapshot date
NOW = pd.datetime(2011,11,1)

# Calculate recency by subtracting current date from the latest InvoiceDate
features = online_X.groupby('CustomerID').agg({
  'Date': lambda x: (NOW - x.max()).days,
  # Calculate frequency by counting unique number of invoices
  'InvoiceNo': pd.Series.nunique,
  # Calculate monetary value by summing all spend values
  'invoice_value': np.sum,
  # Calculate average and total quantity
  'Quantity': ['mean', 'sum']}).reset_index()

# Rename the columns
features.columns = ['CustomerID', 'recency', 'frequency', 'monetary', 'quantity_avg', 'quantity_total']
features.info()

In [ ]:
#Build pivot table with monthly transaction per customer 
cust_month_tx = pd.pivot_table(data = df2, index = ['CustomerID'], 
                              values = 'InvoiceNo', columns = ['Month'], 
                              aggfunc = pd.Series.nunique, fill_value = 0)

cust_month_tx.info

In [ ]:
#Flat the pivot table
flattened = pd.DataFrame(cust_month_tx.to_records())

#Left join Flattened pivot table to the features table on CustomerID
join = features.merge(flattened, on = 'CustomerID')
join

In [ ]:
#Finalise data preparation and split to train/test

#Store identifier and target variable column names
custid = ['CustomerID']
target = ['11']
#Extract target variable 
Y = join[target]

#Extract features for culumns name
cols = [col for col in features.columns if col not in custid]

#Store features
X = features[cols]

In [ ]:
from sklearn.model_selection import train_test_split
#Split data to training and testing
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.25, random_state = 99)

#Print the shape of the datasets
print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape )

In [ ]:
from sklearn.linear_model import LinearRegression
# Initialize linear regression instance
linreg = LinearRegression()

# Fit the model to training dataset
linreg.fit(train_X, train_Y)

# Predict the target variable for training data
train_pred_Y = linreg.predict(train_X)

# Predict the target variable for testing data
test_pred_Y = linreg.predict(test_X)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
# Calculate root mean squared error on training data
rmse_train = np.sqrt(mean_squared_error(train_Y, train_pred_Y))

# Calculate mean absolute error on training data
mae_train = mean_absolute_error(train_Y, train_pred_Y)

# Calculate root mean squared error on testing data
rmse_test = np.sqrt(mean_squared_error(test_Y, test_pred_Y))

# Calculate mean absolute error on testing data
mae_test = mean_absolute_error(test_Y, test_pred_Y)

# Print the performance metrics
print('RMSE train: {}; RMSE test: {}\nMAE train: {}, MAE test: {}'.format(rmse_train, rmse_test, mae_train, mae_test))

In [ ]:
# Import `statsmodels.api` module
import statsmodels.api as sm

# Initialize model instance on the training data
olsreg = sm.OLS(train_Y, train_X)

# Fit the model
olsreg = olsreg.fit()

# Print model summary
print(olsreg.summary())